In [1]:
import sys
sys.dont_write_bytecode = True

# for debug print
def chkprint(*args):
    from inspect import currentframe
    names = {id(v):k for k,v in currentframe().f_back.f_locals.items()}
    print(', '.join(names.get(id(arg),'???')+' : '+str(type(arg))+' = '+repr(arg) for arg in args))

In [2]:
import pandas as pd
import numpy as np
from sklearn import datasets
from sklearn.ensemble        import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
import time

In [3]:
# 参考
# https://di-acc2.com/programming/python/8465/ 

# 時間計測 start
start = time.time()

# ワイン白データ読み込み
wine = pd.read_csv("winequality-white.csv", sep=";", encoding="utf-8")

# データをラベルとデータに分離 
y = wine["quality"]
x = wine.drop("quality", axis=1)

# y のラベルを付け直す 
newlist = []
for v in list(y):
    if v <= 4:
        newlist += [0]
    elif v <=7:
        newlist += [1]
    else:
        newlist += [2]
y = newlist

# 学習用とテスト用に分離する 
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, train_size = 0.8, shuffle = True)

# グリッドサーチによるハイパーパラメータ探索候補設定
# 今回はRandomForestClassifierのパラメータが対象(値は任意に設定)
parameters = {  
    'n_estimators': [10, 20, 30, 50, 100, 300],     # 用意する決定木モデルの数
    'max_features': ('sqrt', 'log2','auto', None),  # ランダムに指定する特徴量の数
    'max_depth':    (10, 20, 30, 40, 50, None),     # 決定木のノード深さの制限値
}

# モデルインスタンス
model = RandomForestClassifier()

# ハイパーパラメータチューニング(グリッドサーチのコンストラクタにモデルと辞書パラメータを指定)
gridsearch = GridSearchCV(estimator = model,        # モデル
                          param_grid = parameters,  # チューニングするハイパーパラメータ
                          scoring = "accuracy"      # スコアリング
                         )

# 演算実行
gridsearch.fit(x_train, y_train)

# グリッドサーチの結果から得られた最適なパラメータ候補を確認
print("最適なパラメータ = ", format(gridsearch.best_params_)) 
print("評価時の正解率 = ", format(gridsearch.best_score_))


# 時間計測 finish
elapsed_time = time.time() - start
print ("\n elapsed_time:{0}".format(elapsed_time) + "[sec]")

最適なパラメータ =  {'max_depth': 30, 'max_features': 'auto', 'n_estimators': 100}
評価時の正解率 =  0.9412962311361326

 elapsed_time:374.21263217926025[sec]
